In [1]:
# list of supported layers
from tensorflow.keras.layers import (
    Input,
    Activation,
    AveragePooling2D,
    BatchNormalization,
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    GlobalAveragePooling2D,
    GlobalMaxPooling2D,
    Lambda, # only for polynomial activation in the form of `Lambda(lambda x: x**2+x)`
    MaxPooling2D,
    ReLU,
    Softmax,
    )
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
# load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
# convert y_train and y_test to one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [4]:
# reshape X_train and X_test to 4D tensor
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [5]:
inputs = Input(shape=(28,28,1))
out = Conv2D(4, 3, use_bias=False, strides=2)(inputs)
out = BatchNormalization()(out)
out = Activation('relu')(out)
out = MaxPooling2D()(out)
out = Conv2D(8, 3, use_bias=True)(out)
out = ReLU()(out)
out = AveragePooling2D()(out)
out = Flatten()(out)
out = Dropout(0.5)(out)
out = Dense(10, activation="softmax")(out)
model = Model(inputs, out)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 13, 13, 4)         36        
                                                                 
 batch_normalization (BatchN  (None, 13, 13, 4)        16        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 13, 13, 4)         0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 6, 6, 4)          0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 8)           296   

In [7]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['acc']
    )

In [8]:
model.fit(X_train, y_train, epochs=15, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/15


2022-12-15 00:58:25.705348: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


469/469 [==============================] - 3s 6ms/step - loss: 1.9912 - acc: 0.2854 - val_loss: 1.1690 - val_acc: 0.7212
Epoch 2/15
469/469 [==============================] - 3s 7ms/step - loss: 1.2871 - acc: 0.5587 - val_loss: 0.7106 - val_acc: 0.8303
Epoch 3/15
469/469 [==============================] - 4s 8ms/step - loss: 1.0419 - acc: 0.6463 - val_loss: 0.5290 - val_acc: 0.8699
Epoch 4/15
469/469 [==============================] - 3s 6ms/step - loss: 0.9145 - acc: 0.6946 - val_loss: 0.4789 - val_acc: 0.8805
Epoch 5/15
469/469 [==============================] - 3s 6ms/step - loss: 0.8584 - acc: 0.7122 - val_loss: 0.4252 - val_acc: 0.8913
Epoch 6/15
469/469 [==============================] - 3s 6ms/step - loss: 0.8263 - acc: 0.7216 - val_loss: 0.4010 - val_acc: 0.8955
Epoch 7/15
469/469 [==============================] - 2s 5ms/step - loss: 0.7976 - acc: 0.7315 - val_loss: 0.3953 - val_acc: 0.8986
Epoch 8/15
469/469 [==============================] - 3s 6ms/step - loss: 0.7841 - acc:

In [9]:
model.save('alt_model.h5')

In [10]:
X = X_test[[0]]
y = model.predict(X)
y

1/1 [==============================] - 0s 44ms/step


array([[1.04527235e-05, 7.22984405e-05, 7.09750550e-03, 1.99398728e-05,
        5.22034138e-10, 4.70989514e-09, 3.12637624e-11, 9.92796242e-01,
        2.48173905e-07, 3.44563432e-06]], dtype=float32)

In [11]:
for layer in model.layers:
    print(layer.__class__.__name__, layer.get_config())
    try:
        print(layer.get_config()['function'])
    except:
        pass
    print(layer.get_input_shape_at(0),layer.get_output_shape_at(0))
    try:
        print(layer.get_weights()[0].shape)
        print(layer.get_weights()[1].shape)
    except:
        pass

InputLayer {'batch_input_shape': (None, 28, 28, 1), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_1'}
(None, 28, 28, 1) (None, 28, 28, 1)
Conv2D {'name': 'conv2d', 'trainable': True, 'dtype': 'float32', 'filters': 4, 'kernel_size': (3, 3), 'strides': (2, 2), 'padding': 'valid', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'groups': 1, 'activation': 'linear', 'use_bias': False, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
(None, 28, 28, 1) (None, 13, 13, 4)
(3, 3, 1, 4)
BatchNormalization {'name': 'batch_normalization', 'trainable': True, 'dtype': 'float32', 'axis': ListWrapper([3]), 'momentum': 0.99, 'epsilon': 0.001, 'center': True, 'scale': True, 'beta_initializer': {'class_name': 'Zeros', 'config': {}}, 'gamma_initializer

In [12]:
import json


In [13]:
with open("alt_model.json", "w") as f:
    json.dump({'X': X.flatten().tolist(), 'y': y.flatten().tolist()}, f)